In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn import preprocessing
import seaborn as sns
from random import shuffle
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
train=pd.read_csv("../input/mobile-price-classification/train.csv")
test=pd.read_csv("../input/mobile-price-classification/test.csv")



In [ ]:
print("Any missing sample in training set:",train.isnull().values.any())
print("Any missing sample in test set:",test.isnull().values.any(), "\n")

In [ ]:
train=pd.read_csv("../input/mobile-price-classification/train.csv")
train.info()

In [ ]:
train.describe()

In [ ]:
train.price_range.value_counts().plot(kind="pie")

In [ ]:
train['price_range'].describe(), train['price_range'].unique()

In [ ]:
def testing_model(x_train, x_test, y_train, y_test, model):
    model.fit(x_train, y_train)
    y_predict = model.predict(x_test)

    conv_matrix = confusion_matrix(y_test, y_predict)
    sns.heatmap(conv_matrix, cmap= 'cubehelix', annot=True, fmt='.0f' )
    print('Confusion matrix \n')
    plt.show()
    
    print('Classification report \n')
    print(classification_report(y_test, y_predict))



In [ ]:
y = train["price_range"].values
x = train.drop(["price_range"],axis=1)
x= (x-np.min(x))/(np.max(x)-np.min(x))

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state=1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_t = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)
print(x_train.shape)

In [ ]:
svm_model1=SVC(C=100,decision_function_shape="ovo",gamma="auto",kernel="linear",random_state=1)
svm_model1.fit(x_train,y_train)
testing_model(x_train, x_test, y_train, y_test, svm_model1)
print("train_accuracy:",svm_model1.score(x_train,y_train))
print("test_accuracy: ", svm_model1.score(x_test,y_test))

In [ ]:
np.unique(y_train, return_counts=True)

In [ ]:
np.unique(y_test, return_counts=True)

In [ ]:
C=[1,0.1,0.25,0.5,2,8,10,100]
kernel=["linear","rbf","poly","sigmoid"]
gamma=["auto",0.01,0.001,0.0001,1]
decision_function_shape=["ovo","ovr"]

In [ ]:
svm=SVC(random_state=1)
grid_svm=GridSearchCV(estimator=svm,cv=5,param_grid=dict(kernel=kernel,C=C, gamma=gamma, decision_function_shape=decision_function_shape))
grid_svm.fit(x_train,y_train)
print("best score: ", grid_svm.best_score_)
print("best param: ", grid_svm.best_params_)


In [ ]:
svm_model=SVC(C=100,decision_function_shape="ovo",gamma="auto",kernel="linear",random_state=1)
svm_model.fit(x_train,y_train)
testing_model(x_train, x_test, y_train, y_test, svm_model)
print("train_accuracy:",svm_model.score(x_train,y_train))
print("valid_accuracy: ", svm_model.score(x_test,y_test))

In [ ]:
pca = PCA(0.95)
pca.fit(x)
pca_features = pca.transform(x)

x_train, x_valid, y_train, y_valid = train_test_split(pca_features, y, test_size = 0.3, random_state=10) 

scaler =  preprocessing.StandardScaler()
scaler.fit(pca_features)
norm_x_train = scaler.transform(x_train)
norm_x_test = scaler.transform(x_valid)
print(x_train.shape)
svm_model1=SVC(C=100,decision_function_shape="ovo",gamma="auto",kernel="linear",random_state=1)
svm_model1.fit(x_train,y_train)
testing_model(x_train, x_valid, y_train, y_valid, svm_model1)
print("train_accuracy:",svm_model1.score(x_train,y_train))
print("valid_accuracy: ", svm_model1.score(x_valid,y_valid))